In [ ]:
pip install tensorflow numpy opencv-python

In [ ]:
!unzip ruedas.zip

In [5]:
import os
# Leer las imágenes de la carpeta
path = "/content/ruedas"
image_files = sorted(os.listdir(path))

In [6]:
# Definir la longitud de la secuencia
sequence_length = 5

# Agrupar las imágenes en secuencias
sequences = []
labels = []

In [8]:
import cv2
import numpy as np

# Ordenar los nombres de los archivos de imagen por video_name, wheel_number y frame_number
image_files.sort(key=lambda x: (x.split("_")[0], int(x.split("_")[2]), int(x.split("_")[4])))

# Recorrer las imágenes
for i in range(sequence_length, len(image_files)):
    # Comprobar si las imágenes consecutivas pertenecen al mismo video y a la misma rueda
    if image_files[i].split("_")[0] == image_files[i-sequence_length].split("_")[0] and image_files[i].split("_")[2] == image_files[i-sequence_length].split("_")[2]:
        # Crear una secuencia de 'sequence_length' imágenes
        sequence = [cv2.imread(os.path.join(path, image_files[j]), cv2.IMREAD_GRAYSCALE) for j in range(i-sequence_length, i)]
        # Redimensionar, normalizar y aplanar las imágenes de la secuencia
        sequence = [cv2.resize(image, (32, 32)) / 255.0 for image in sequence]
        sequence = [image.flatten() for image in sequence]
        # Añadir la secuencia a la lista de secuencias
        sequences.append(sequence)
        # Añadir la etiqueta correspondiente a la lista de etiquetas
        labels.append(int(image_files[i].split("_")[-1].split(".")[0]))

# Convertir las secuencias y las etiquetas en arrays de numpy
X = np.array(sequences)
y = np.array(labels)

In [9]:
#balancear clases:

# Obtener los índices de las clases minoritaria y mayoritaria
minority_class_indices = np.where(y == 0)[0]
majority_class_indices = np.where(y == 1)[0]

# Calcular el número de ejemplos para cada clase
num_samples = min(len(minority_class_indices), len(majority_class_indices))

# Mantener solo los primeros 'num_samples' ejemplos de cada clase
minority_class_indices = minority_class_indices[:num_samples]
majority_class_indices = majority_class_indices[-num_samples:]

# Concatenar los índices de ambas clases
balanced_indices = np.concatenate((minority_class_indices, majority_class_indices))

# Usar estos índices para seleccionar las secuencias y las etiquetas correspondientes
X = X[balanced_indices]
y = y[balanced_indices]


In [11]:
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.regularizers import l1_l2
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
#definimos el modelo
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(5, 32*32), kernel_regularizer=l1_l2(l1=0.0001, l2=0.0001)))
model.add(Dropout(0.5))  # Aumentar dropout al 50%
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Definir la parada temprana
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)



In [13]:

# Entrenar el modelo
model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test), callbacks=[es])


Epoch 1/500
20/20 [==============================] - 2s 36ms/step - loss: 1.6737 - accuracy: 0.7052 - val_loss: 1.2335 - val_accuracy: 0.8506
Epoch 2/500
20/20 [==============================] - 0s 11ms/step - loss: 1.0932 - accuracy: 0.8681 - val_loss: 0.9050 - val_accuracy: 0.8701
Epoch 3/500
20/20 [==============================] - 0s 12ms/step - loss: 0.7960 - accuracy: 0.8860 - val_loss: 0.6987 - val_accuracy: 0.9351
Epoch 4/500
20/20 [==============================] - 0s 11ms/step - loss: 0.6239 - accuracy: 0.9511 - val_loss: 0.5410 - val_accuracy: 0.9610
Epoch 5/500
20/20 [==============================] - 0s 12ms/step - loss: 0.5106 - accuracy: 0.9577 - val_loss: 0.4839 - val_accuracy: 0.9545
Epoch 6/500
20/20 [==============================] - 0s 11ms/step - loss: 0.4580 - accuracy: 0.9723 - val_loss: 0.3880 - val_accuracy: 0.9805
Epoch 7/500
20/20 [==============================] - 0s 13ms/step - loss: 0.3755 - accuracy: 0.9902 - val_loss: 0.3291 - val_accuracy: 1.0000
Epoch 

In [14]:
# Guardar el modelo
model.save('LSTM.h5')



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
